In [2]:
import os
import sys
import glob
import argparse
import matplotlib.pyplot as plt

from keras import __version__ 
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD


In [3]:
IM_WIDTH, IM_HEIGHT = 299, 299
NB_EPOCHS = 3
BAT_SIZE = 32
FC_SIZE = 1024
NB_IV3_LAYERS_TO_FREEZE = 172

In [4]:
def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + '/*')))
    return cnt


In [5]:
get_nb_files('/home/bohdan/xray_project/data/train/')

24

In [7]:
def setup_to_transfer_learn(model, base_model):
    '''
    Freeze all layers and compile the model
    '''
    for layer in base_model.layers:
        layer.trainable = False
        
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy']
                 )

In [ ]:
def add_new_last_layer(base_model, nb_classes):
    '''
    Add last layer to the convnet
    
    Args:
         base_model: keras model excluding top
         nb_classes: # of classes
         
    Return:
         new keras model with last layer
    '''
    x = base_model.output
    x = GlobalAveragePooling2D()(x) # new FC layer, random init
    x = Dense(FC_SIZE, activation='relu')(x) # new softmax layer
    predictions = Dense(nb_classes)
    model = Model(input=base_model.input, output=predictions)
    return model